In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preprocessing

In [25]:
#Acces Dataset
data = '/content/drive/MyDrive/Portfolio/Chatbot/Dataset/intents.json'

In [26]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Libraries needed for Tensorlflo
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
with open(data) as json_data:
  intents = json.load(json_data)

intents

{'intents': [{'tag': 'salam',
   'patterns': ['Halo',
    'Apa kabar?',
    'Ada yang di sana?',
    'Hai',
    'Selamat pagi'],
   'responses': ['Halo, terima kasih sudah berkunjung',
    'Senang bertemu denganmu lagi',
    'Hai, bagaimana saya bisa membantu?',
    'Halo selamat datang!'],
   'context_set': ''},
  {'tag': 'pertanyaan_nama',
   'patterns': ['Siapa nama kamu?',
    'Boleh tahu namamu?',
    'Apa nama lengkapmu?',
    'Kamu punya nama nggak?'],
   'responses': ['Saya adalah Andri Rahmadani, senang bertemu denganmu!',
    'Anda bisa memanggil saya Andri Rahmadani.',
    'Nama lengkap saya adalah Andri Rahmadani.']},
  {'tag': 'pertanyaan_seputar_hobi',
   'patterns': ['Apa hobi kamu?',
    'Hobi apa yang kamu suka?',
    'Boleh tahu apa yang kamu sukai?',
    'Apa kegemaranmu?'],
   'responses': ['Saya menyukai banyak hal! Beberapa hobiku antara lain membaca buku, mendengarkan musik, dan bermain olahraga.',
    'Hobiku termasuk membaca, bermain musik, dan mengeksplorasi t

In [28]:
words = []
classes = []
documents = []
ignore = ['?']

# loop through each sentence in the intent's patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)

    documents.append((w, intent['tag']))

    if intent['tag'] not in classes:
      classes.append(intent['tag'])



In [29]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents",  documents)
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

42 documents [(['Halo'], 'salam'), (['Apa', 'kabar', '?'], 'salam'), (['Ada', 'yang', 'di', 'sana', '?'], 'salam'), (['Hai'], 'salam'), (['Selamat', 'pagi'], 'salam'), (['Siapa', 'nama', 'kamu', '?'], 'pertanyaan_nama'), (['Boleh', 'tahu', 'namamu', '?'], 'pertanyaan_nama'), (['Apa', 'nama', 'lengkapmu', '?'], 'pertanyaan_nama'), (['Kamu', 'punya', 'nama', 'nggak', '?'], 'pertanyaan_nama'), (['Apa', 'hobi', 'kamu', '?'], 'pertanyaan_seputar_hobi'), (['Hobi', 'apa', 'yang', 'kamu', 'suka', '?'], 'pertanyaan_seputar_hobi'), (['Boleh', 'tahu', 'apa', 'yang', 'kamu', 'sukai', '?'], 'pertanyaan_seputar_hobi'), (['Apa', 'kegemaranmu', '?'], 'pertanyaan_seputar_hobi'), (['Apa', 'keahlianmu', '?'], 'keahlian'), (['Bidang', 'apa', 'yang', 'kamu', 'kuasai', '?'], 'keahlian'), (['Apa', 'yang', 'bisa', 'kamu', 'lakukan', '?'], 'keahlian'), (['Apa', 'spesialisasimu', '?'], 'keahlian'), (['Apa', 'latar', 'belakang', 'pendidikanmu', '?'], 'pendidikan'), (['Dari', 'universitas', 'mana', 'kamu', '?'], 

In [30]:
training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    patterns_words = doc[0]
    patterns_words = [stemmer.stem(word.lower()) for word in patterns_words]

    # Membuat vektor bag-of-words dengan list comprehension
    bag = [1 if w in patterns_words else 0 for w in words]

    # Membuat vektor output
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Menambahkan pasangan (vektor bag-of-words, vektor output) ke dalam list training
    training.append([bag, output_row])

# Mengacak urutan data pelatihan
random.shuffle(training)

# Mengkonversi list training menjadi array NumPy
training = np.array(training)

# Memisahkan input dan output
train_x = list(training[:, 0])
train_y = list(training[:, 1])


<ipython-input-30-b501d2170467>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [31]:
print(train_x)
print(train_y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Building Model

In [32]:
import tensorflow as tf

model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(10, input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [33]:
model.fit(np.array(train_x), np.array(train_y), epochs = 500, batch_size = 8, verbose = 1)

Epoch 1/500
6/6 [==============================] - 1s 3ms/step - loss: 2.3026 - accuracy: 0.1429
Epoch 2/500
6/6 [==============================] - 0s 3ms/step - loss: 2.2723 - accuracy: 0.1667
Epoch 3/500
6/6 [==============================] - 0s 3ms/step - loss: 2.2483 - accuracy: 0.1667
Epoch 4/500
6/6 [==============================] - 0s 3ms/step - loss: 2.2239 - accuracy: 0.2143
Epoch 5/500
6/6 [==============================] - 0s 3ms/step - loss: 2.2010 - accuracy: 0.2619
Epoch 6/500
6/6 [==============================] - 0s 3ms/step - loss: 2.1799 - accuracy: 0.2857
Epoch 7/500
6/6 [==============================] - 0s 3ms/step - loss: 2.1592 - accuracy: 0.3095
Epoch 8/500
6/6 [==============================] - 0s 3ms/step - loss: 2.1374 - accuracy: 0.3095
Epoch 9/500
6/6 [==============================] - 0s 3ms/step - loss: 2.1172 - accuracy: 0.3333
Epoch 10/500
6/6 [==============================] - 0s 3ms/step - loss: 2.0949 - accuracy: 0.3810
Epoch 11/500
6/6 [===========

In [34]:
model.save("chatbot.pkl")

# Making Prediction

In [35]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [36]:
from keras.models import load_model
model = load_model("chatbot.pkl")

In [37]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [38]:
import numpy as np
import nltk

def clean_up_sentence(sentence):
    # Tokenisasi kalimat
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming setiap kata
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    # Tokenisasi dan stemming kalimat
    sentence_words = clean_up_sentence(sentence)

    # Inisialisasi vektor bag of words
    bag = np.zeros(len(words), dtype=int)

    # Mengisi nilai vektor menggunakan list comprehension
    bag = [1 if s in sentence_words else 0 for s in words]

    return np.array(bag)


In [39]:
ERROR_THRESHOLD = 0.30

def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results[0]) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [(classes[r[0]], r[1]) for r in results]
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # find a tag matching the first result
        for i in intents['intents']:
            if i['tag'] == results[0][0]:
                # a random response from the intent
                return random.choice(i['responses'])
    # return a default response or message indicating lack of understanding
    return "Maap aku tidak mengerti!."


# Let's start conversation

In [41]:
response('Hallo')

1/1 [==============================] - 0s 29ms/step


'Halo selamat datang!'

In [42]:
response('Siapa namamu?')

1/1 [==============================] - 0s 27ms/step


'Anda bisa memanggil saya Andri Rahmadani.'

In [45]:
response('Dimana tempat tinggalmu?')

1/1 [==============================] - 0s 24ms/step


'Saya tinggal di Banjarmasin, Indonesia.'

In [46]:
response('Keahlian apa yang kamu kuasai?')

1/1 [==============================] - 0s 32ms/step


'Saya memiliki keahlian di bidang Machine Learning, Artificial Intelligence, dan Data Science.'